In [5]:
pip install torchvision

Note: you may need to restart the kernel to use updated packages.


In [1]:
import zipfile
import os
files = zipfile.ZipFile('train.zip','r')
files.extractall(os.getcwd())
files.close()

In [ ]:
#201808
#Pytorch 0.4.0
"""
Attention: This code has Exploding Gradients problem
"""

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset
import torchvision.datasets as datasets
import os
from PIL import Image
import torchvision.transforms as transforms



LearningRate = 0.001
EPOCH = 20
BATCH = 128
SAVE_CSV = 'eval.csv'
TEST_DIR = './test1'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class TestImageFolder(torch.utils.data.Dataset):
    def __init__(self, root, transform=None):
        self.imgs = []
        for filename in os.listdir('./test1'):
            if filename.endswith('jpg'):
                self.imgs.append('{}'.format(filename))
        self.root = root
        self.transform = transform

    def __getitem__(self, index):
        filename = self.imgs[index]
        img = Image.open(os.path.join(self.root, filename))
        if self.transform is not None:
            img = self.transform(img)
            return img, filename

    def __len__(self):
        return len(self.imgs)
    

# Data
transform_train = transforms.Compose([
        transforms.Resize(256),
    transforms.RandomResizedCrop(227),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
train_dataset = datasets.ImageFolder(root='./train/', transform=transform_train)
print(train_dataset.class_to_idx)

NUM_classes = len(train_dataset.classes)
print('number of class: %d' %NUM_classes)

train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=BATCH,
                                           num_workers=0,
                                           pin_memory=True,
                                           )
print('==>>> total training batch number: {}'.format(len(train_loader)))

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])
test_dataset = TestImageFolder(TEST_DIR, transform_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH, 
                                          shuffle=False, num_workers=0)

print('==>>> total testing batch number: {}'.format(len(test_loader)))
print('========================================')


#Local Response Normalization(LRN)
class LRN(nn.Module):
    def __init__(self, local_size=1, alpha=1.0, beta=0.75, ACROSS_CHANNELS=False):
        super(LRN, self).__init__()
        self.ACROSS_CHANNELS = ACROSS_CHANNELS
        if self.ACROSS_CHANNELS:
            self.average=nn.AvgPool3d(kernel_size=(local_size, 1, 1),
                    stride=1,
                    padding=(int((local_size-1.0)/2), 0, 0))
        else:
            self.average=nn.AvgPool2d(kernel_size=local_size,
                    stride=1,
                    padding=int((local_size-1.0)/2))
        self.alpha = alpha
        self.beta = beta

    def forward(self, x):
        if self.ACROSS_CHANNELS:
            div = x.pow(2).unsqueeze(1)
            div = self.average(div).squeeze(1)
            div = div.mul(self.alpha).add(1.0).pow(self.beta)
        else:
            div = x.pow(2)
            div = self.average(div)
            div = div.mul(self.alpha).add(1.0).pow(self.beta)
        x = x.div(div)
        return x

#AlexNet network
class myAlexNet(torch.nn.Module):
    def __init__(self, num_classes = NUM_classes):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 96, 11, stride=4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d((3,3), stride = 2),
            LRN(local_size=5, alpha=1e-4, beta=0.75, ACROSS_CHANNELS=True),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(96, 256, 3, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d((3, 3), stride=2),
            LRN(local_size=5, alpha=1e-4, beta=0.75, ACROSS_CHANNELS=True)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 384, 3, stride=1, padding=1),
            nn.ReLU(inplace=True),
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(384, 384, 3, stride=1, padding=1),
            nn.ReLU(inplace=True),
        )
        self.layer5 = nn.Sequential(
            nn.Conv2d(384, 256, 3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d((3,3), stride=2),
        )
        self.layer6 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256*6*6, 4096),
            nn.ReLU(inplace=True),
        )
        self.layer7 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
        )
        self.layer8 = nn.Linear(in_features=4096, out_features=num_classes)

    def forward(self, x):
        x = self.layer5(self.layer4(self.layer3(self.layer2(self.layer1(x)))))
        x = x.view(x.size()[0], -1)
        x = self.layer8(self.layer7(self.layer6(x)))
        return x

    def name(self):
        return "AlexNet"

alexnet = myAlexNet().to(device)
#print(alexnet)
criterion = nn.CrossEntropyLoss(reduction='sum')
optimizer = optim.SGD(alexnet.parameters(), lr=LearningRate, momentum=0.9)

if __name__ == "__main__":
    #training step
    for epoch in range(EPOCH):
        running_loss = 0.0
        running_acc = 0.0
        for i, data in enumerate(train_loader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            output = alexnet(inputs)
            loss = criterion(output, labels)

            # backward
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 39 == 0:
                print('[%d, %d] loss: %.03f'
                      % (epoch + 1, i + 1, running_loss / 100))
                running_loss = 0.0   

    with torch.no_grad():
        results = []
        nameid = []

        for ii, data in enumerate(test_loader):
            images, filename = data
            images = images.to(device)
            outputs = alexnet(images)
            _, predicted = torch.max(outputs.data, 1)
            results.extend(predicted)
            nameid.extend(filename)                  

        # write a csv file
        f = open(SAVE_CSV, "w")
        for i in range(len(nameid)):
            f.write("{} {}\n".format(nameid[i], results[i]))
        f.close()

{'cat': 0, 'dog': 1}
number of class: 2
==>>> total training batch number: 196
==>>> total testing batch number: 98
[1, 1] loss: 0.878
[1, 40] loss: 1.355
[1, 79] loss: 0.000
[1, 118] loss: 29947.115
[1, 157] loss: 0.000
[1, 196] loss: 0.000
[2, 1] loss: 107215.270
[2, 40] loss: nan
[2, 79] loss: nan
[2, 118] loss: nan
[2, 157] loss: nan
[2, 196] loss: nan
[3, 1] loss: nan
[3, 40] loss: nan
[3, 79] loss: nan
[3, 118] loss: nan
[3, 157] loss: nan
[3, 196] loss: nan
[4, 1] loss: nan
[4, 40] loss: nan
[4, 79] loss: nan
[4, 118] loss: nan
[4, 157] loss: nan
[4, 196] loss: nan
[5, 1] loss: nan
[5, 40] loss: nan
[5, 79] loss: nan
[5, 118] loss: nan
[5, 157] loss: nan
[5, 196] loss: nan
[6, 1] loss: nan
[6, 40] loss: nan
[6, 79] loss: nan
[6, 118] loss: nan
[6, 157] loss: nan
[6, 196] loss: nan
[7, 1] loss: nan
[7, 40] loss: nan
[7, 79] loss: nan
[7, 118] loss: nan
[7, 157] loss: nan
[7, 196] loss: nan
[8, 1] loss: nan
[8, 40] loss: nan
[8, 79] loss: nan
[8, 118] loss: nan
[8, 157] loss: nan
